In [0]:
# SECRET SCOPE NAMES CREATED FOR L AND P AZURE SECRETS follow this format
# azure_kv : secret_scope_name

# kv-sw-ds-p-era-rw-001 : azure_kv_p_001
# kv-sw-ds-l-era-rw-001 : azure_kv_l_001

# EXAMPLE
# azure_kv_p_001 : sf-p-user-001
# azure_kv_p_001 : sf-p-pwd-001

# azure_kv_l_001 : sf-l-user-001
# azure_kv_l_001 : sf-l-pwd-001

# # Use dbutils secrets to get Snowflake credentials.
# dbutils.secrets.list("Key_vault_secrets")
# dbutils.secrets.list(scope="azure_kv_p_001")
# user = dbutils.secrets.get(scope="azure_kv_p_001", key="sf-p-user-001")
# password = dbutils.secrets.get(scope="azure_kv_p_001", key="sf-p-pwd-001")

In [0]:
# SF QUERY THAT CONTAINS ALL POSSIBLE COLUMN NAMES ASSOCIATED WITH LTF
# query_00 = sfdt.sql("select \
#                     CUSTOMER.DIVISION \
#                     , trim(Customer.KEY_ACCOUNT_PLAN_CP3) as KA_PLAN \
#                     , trim(Customer.KEY_ACCOUNT_PLAN_DESCRIPTION_CP3) as KA_PLAN_DESC \
#                     , Customer.CUSTOMER_NUMBER \
#                     , Customer.SALES_OFFICE_DESCRIPTION \
#                     , Customer.DISTRIBUTION_CHANNEL_DESCRIPTION \
#                     , trim(Customer.CUSTOMER_TRADE_CHANNEL_DESCRIPTION) as CUSTOMER_TRADE_CHANNEL_DESCRIPTION\
#                     , Customer.SUPER_CHANNEL \
#                     , Customer.SUPER_CHANNEL_DEFINITION \
#                     , Customer.ADDRESS_ZIP_CODE as ZIP_CODE \
#                     , DELIVERY_EXECUTION.ACTUAL_GOODS_MOVEMENT_DATE as ACTUAL_DATE \
#                     , trim(DELIVERY_EXECUTION.MATERIAL_ID) as DE_MATERIAL_ID \
#                     , DELIVERY_EXECUTION.ORDERED \
#                     , DELIVERY_EXECUTION.LOADED \
#                     , trim(DELIVERY_EXECUTION.DELIVERED) as DELIVERED \
#                     , DELIVERY_EXECUTION.RETURNED \
#                     , DELIVERY_EXECUTION.REJECTED \
#                     , DELIVERY_EXECUTION.DAMAGED \
#                     , trim(DELIVERY_EXECUTION.OUT_OF_STOCK) as OUT_OF_STOCK \
#                     , DELIVERY_EXECUTION.RETURN_TO_STOCK \
#                     , DELIVERY_EXECUTION.PRODUCT_NOT_ON_TRUCK \
#                     , DELIVERY_EXECUTION.SOLDTO_CUSTOMER_ID \
#                     , trim(DELIVERY_EXECUTION.SHIPTO_CUSTOMER_ID) as DE_SHIPTO_CUSTOMER_ID \
#                     , DELIVERY_EXECUTION.SALES_OFFICE_ID \
#                     , trim(DELIVERY_EXECUTION.SALES_ORGANIZATION) as DE_SALES_ORG \
#                     , trim(DELIVERY_EXECUTION.PROFIT_CENTER_ID) as PROFIT_CENTER_ID \
#                     , DELIVERY_EXECUTION.DISTRIBUTION_CHANNEL_ID \
#                     , trim(DELIVERY_EXECUTION.FISCAL_WEEK) as FISCAL_WEEK \
#                     , trim(DELIVERY_EXECUTION.FISCAL_YEAR) as FISCAL_YEAR \
#                     , trim(DELIVERY_EXECUTION.FISCAL_QUARTER) as FISCAL_QUARTER \
#                     , trim(DELIVERY_EXECUTION.FISCAL_PERIOD) as FISCAL_PERIOD \
#                     , MATERIAL.BRAND_DESC as BRAND_DESC \
#                     , MATERIAL.EA_UPC as EA_UPC \
#                     , trim(MATERIAL.PACK_SIZE_DESC) as PACK_SIZE_DESC \
#                     , trim(MATERIAL.FLAVOUR_DESC) as FLAVOR_DESC \
#                     , MATERIAL.PACK_TYPE as PACK_TYPE \
#                     , MATERIAL.ACCT_ASSMT_GRP_MATL_DESC as ACCT_ASSIGNMENT_GRP_MAT_DESC \
#                     , MATERIAL.SWIRE_SWEEP_DESCRIPTION \
#                     from \
#                     db_bi_p_edw.general_use_bas.bas_fact_delivery_execution as DELIVERY_EXECUTION \
#                     inner join \
#                     DB_SWIRE_BI_P_EDW.TRANSFORMED.DIM_MDM_CUSTOMER_MASTER as CUSTOMER on (DELIVERY_EXECUTION.SOLDTO_CUSTOMER_ID = CUSTOMER.customer_number) \
#                     inner join \
#                         DB_SWIRE_BI_P_EDW.TRANSFORMED.DIM_MDM_MATERIAL_MASTER as MATERIAL on (DELIVERY_EXECUTION.MATERIAL_SK = MATERIAL.MATERIAL_SK) \
#                     where \
#                     DELIVERY_EXECUTION.ACTUAL_GOODS_MOVEMENT_DATE >= date('2019-01-01') \
#                     and \
#                     DELIVERY_EXECUTION.ACTUAL_GOODS_MOVEMENT_DATE < CURRENT_DATE() \
#                     and \
#                     DELIVERY_EXECUTION.ACTUAL_GOODS_MOVEMENT_DATE != date('9999-12-31') \
#                     and \
#                     MATERIAL.MATERIAL_TYPE in ('ZFER') \
#                     and \
#                     trim(CUSTOMER.DIVISION) = 'SA'")

In [0]:
# IMPORTING DATA ANALYSIS AND VISUALIZATION LIBRARIES
import pandas as pd
pd.set_option('display.max_columns', None)

# WARNINGS AND LOGISTICS 
#   Make sure we do not get line breaks when doing show on wide dataframes
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
import warnings
warnings.filterwarnings("ignore")
#   disable informational messages from prophet
logging.getLogger('py4j').setLevel(logging.ERROR)



/databricks/python/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [0]:
%run ./Snowflake_Connection

In [0]:
sfdt = SnowflakeDataTool()

In [0]:
sf_demand_00 = sfdt.sql("select \
                    trim(DELIVERY_EXECUTION.ACTUAL_GOODS_MOVEMENT_DATE) as ACTUAL_DATE \
                    , trim(DELIVERY_EXECUTION.MATERIAL_ID) as MATERIAL_ID \
                    , trim(DELIVERY_EXECUTION.ORDERED) as ORDERED \
                    , trim(DELIVERY_EXECUTION.DELIVERED) as DELIVERED \
                    , trim(DELIVERY_EXECUTION.SHIPTO_CUSTOMER_ID) as CUSTOMER_ID \
                    , trim(DELIVERY_EXECUTION.FISCAL_WEEK) as FISCAL_WEEK \
                    , trim(DELIVERY_EXECUTION.FISCAL_YEAR) as FISCAL_YEAR \
                    from \
                    db_bi_p_edw.general_use_bas.bas_fact_delivery_execution as DELIVERY_EXECUTION \
                    inner join \
                        DB_SWIRE_BI_P_EDW.TRANSFORMED.DIM_MDM_MATERIAL_MASTER as MATERIAL on (DELIVERY_EXECUTION.MATERIAL_SK = MATERIAL.MATERIAL_SK) \
                    where \
                    DELIVERY_EXECUTION.ACTUAL_GOODS_MOVEMENT_DATE >= date('2019-01-01') \
                    and \
                    DELIVERY_EXECUTION.ACTUAL_GOODS_MOVEMENT_DATE < CURRENT_DATE() \
                    and \
                    DELIVERY_EXECUTION.ACTUAL_GOODS_MOVEMENT_DATE != date('9999-12-31') \
                    and \
                    MATERIAL.MATERIAL_TYPE in ('ZFER')")

display(sf_demand_00.orderBy('ACTUAL_DATE'))

sf_demand_00.write.mode('overwrite').saveAsTable('ltf_db.ltf_raw_demand_sf_query')

ACTUAL_DATE,MATERIAL_ID,ORDERED,DELIVERED,CUSTOMER_ID,FISCAL_WEEK,FISCAL_YEAR
2019-01-01,000000000000121750,8.000,8.000,0600267076,01,2019
2019-01-01,000000000000102280,0.000,-1.000,0600263853,01,2019
2019-01-01,000000000000114756,1.000,1.000,0600251868,01,2019
2019-01-01,000000000000132606,20.000,0.000,0600250119,01,2019
2019-01-01,000000000000132532,1.000,1.000,0600264974,01,2019
2019-01-01,000000000000112259,1.000,1.000,0600252770,01,2019
2019-01-01,000000000000154656,1.000,1.000,0600251763,01,2019
2019-01-01,000000000000152202,1.000,1.000,0600250946,01,2019
2019-01-01,000000000000135573,1.000,1.000,0600250180,01,2019
2019-01-01,000000000000154548,1.000,0.000,0600263897,01,2019
